<a href="https://colab.research.google.com/github/jenriver/jax-bonsai/blob/qwen3/bonsai/models/qwen3/qwen3_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suggested runtime: TPU v2-8

In [1]:
!pip install setuptools
!pip install -q ml-dtypes
!pip install -q kagglehub
!pip install -q tensorboardX
!pip install -q grain
!pip install -q git+https://github.com/google/tunix

!pip uninstall -q -y flax
!pip install -q git+https://github.com/google/flax.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.3/479.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 MB 13.4 MB/s eta 0:00:00


In [2]:
!pip uninstall -q -y jax-bonsai
!pip install git+https://github.com/jenriver/jax-bonsai@qwen3

  Cloning https://github.com/jenriver/jax-bonsai (to revision qwen3) to /tmp/pip-req-build-oh4n2v0v
  Running command git clone --filter=blob:none --quiet https://github.com/jenriver/jax-bonsai /tmp/pip-req-build-oh4n2v0v
  Running command git checkout -b qwen3 --track origin/qwen3
  Switched to a new branch 'qwen3'
  Branch 'qwen3' set up to track remote branch 'qwen3' from 'origin'.
  Resolved https://github.com/jenriver/jax-bonsai to commit ca6eb72a16c0f4cbfe1ed7e853d930f9bde4c2aa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jax-bonsai: filename=jax_bonsai-0.0.0-py3-none-any.whl size=29177 sha256=684a1f33d67533c78314dbacef45c73000023083b52887bbd4fa2b5771b4eadf
  Stored in directory: /tmp/pip-ephem-wheel-cache-znhlean4/wheels/d6/b7/55/8db8eb5d4026dc5a3658e35f258fb0953544ad0fadc325b3fc
Successfully built jax-bonsai


In [3]:
from pip._internal.commands.show import search_packages_info as search_packages_info

package_name='jax-bonsai'
package_info=next(search_packages_info([package_name]))
print(package_info)

_PackageInfo(name='jax-bonsai', version='0.0.0', location='/usr/local/lib/python3.11/dist-packages', editable_project_location=None, requires=['flax', 'jax', 'jaxlib'], required_by=[], installer='pip', metadata_version='2.4', classifiers=['Development Status :: 3 - Alpha', 'Intended Audience :: Developers', 'Intended Audience :: Science/Research', 'Programming Language :: Python :: 3.10', 'Programming Language :: Python :: 3.11', 'Programming Language :: Python :: 3.12', 'Programming Language :: Python :: 3.13', 'Topic :: Scientific/Engineering :: Artificial Intelligence'], summary='A collection of lightweight, open-source models built on JAX.', homepage='', project_urls=['Source, https://github.com/jenriver/jax-bonsai'], author='', author_email='JAX-bonsai developers <jiyounha@google.com>', license='', entry_points=[], files=['bonsai/generate/__pycache__/attention_utils.cpython-311.pyc', 'bonsai/generate/__pycache__/beam_search.cpython-311.pyc', 'bonsai/generate/__pycache__/contrastiv

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

directory_path = '/content/drive/MyDrive/colab-mount'
directory_files = os.listdir(directory_path)
print(directory_files)

['qwen-3-transformers-0.6b-v1.tar.gz', 'test.gdoc', 'qwen-3-transformers-0.6b-v1']


In [6]:
from bonsai.models.qwen3 import params
from bonsai.models.qwen3 import model
from flax import nnx

MODEL_CP_PATH = '/content/drive/MyDrive/colab-mount/qwen-3-transformers-0.6b-v1'

config = model.ModelConfig.qwen3_0_6_b()  # pick correponding config based on model version
qwen3 = params.create_model_from_safe_tensors(MODEL_CP_PATH, config)
nnx.display(qwen3)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen3ForCausalLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_CP_PATH)

In [8]:
from bonsai.generate import sampler

def templatize(prompts):
  out = []
  for p in prompts:
    out.append(
        tokenizer.apply_chat_template(
          [
              {"role": "user", "content": p},
          ],
          tokenize=False,
          add_generation_prompt=True,
          enable_thinking=True,
      )
    )
  return out

inputs = templatize(
    [
        "which is larger 9.9 or 9.11?",
        "Why is the sky blue?",
        "How do you say cheese in French?",
    ]
)

sampler = sampler.Sampler(qwen3, tokenizer, sampler.CacheConfig(cache_size=256, num_layers=28, num_kv_heads=8, head_dim=128))
out = sampler(inputs, total_generation_steps=128, echo=True)

for t in out.text:
  print(t)
  print('*' * 30)


<|im_start|>user
which is larger 9.9 or 9.11?<|im_end|>
<|im_start|>assistant
<think>
Okay, so I need to figure out which number is larger between 9.9 and 9.11. Let me think. Both numbers are in decimal form, right? 9.9 and 9.11. Hmm, decimal numbers can be tricky sometimes, but I remember that when comparing decimals, you can look at the digits from left to right, starting with the first non-zero digit. 

First, let me write them down to visualize better: 9.9 and 9.11. Both start with a 9. So, the first digit after the decimal is the tenths place
******************************
<|im_start|>user
Why is the sky blue?<|im_end|>
<|im_start|>assistant
<think>
Okay, the user is asking why the sky is blue. I need to explain this in a clear and accurate way. First, I should mention the basic reason, which is the scattering of sunlight by the Earth's atmosphere. Then, I should explain the Rayleigh scattering theory. Maybe include some examples, like how different colors are scattered in differe